<p style="font-weight:bold;"> <span style="font-size: 36px"> Import Cashflows Specifications </span> </p>

The aim of this notebook is to document the calculation of the Present Values and Amortization Factors starting from the nominal Cashflows and the Yearly Yield Curve. These starting data is imported from the corresponding csv files located in the following path <code>Initialization/Systemorph/</code>. The process is pictorially explained in the following flowchart. 

<a name="TOC"></a>
<center><img src="../Images/SpecificationsImportCashflows.PNG" width="45%" style="float:center;">

Throughout this notebook we adopt the following variable notation:  we use the words *computed* vs. *expected* to differentiate the results provided by the application vs. this test notebook, respectively.

[Setup IFRS17 Calculation Engine](../Import/ImportScopeCalculation):

In [ ]:
#!import "SpecificationsSetup"

---

# Import Data

<a id='year-and-period'></a>
## Current Period

Please select the target period (i.e. year and month), Reporting Node, Economic Basis, and Data Node based on the imported data:

In [ ]:
var year            = 2021    ;
var month           = 3       ;
var reportingNode   = "CH"    ;
var economicBasis   = "C"     ;

Import the Dimensions, Yield Curves and Nominal Cashflows from the corresponding csv files in <code>Initialization/Systemorph/</code>

In [ ]:
#!eval-notebook "../Initialization/InitSystemorphBaseToMemory"

In [ ]:
Workspace.InitializeFrom(DataSource);

At this point reference data are loaded. Import now a cashfow file with a 720 elements.

In [ ]:
var groupOfContract = "GricComplex";
await UploadCashflowsAsync("Data/CashflowComplex.csv")

Initialization of the Test suite: the data loaded from csv files are ready to be used by the Scopes for the calculations

In [ ]:
var Test = await StartCalculatorAsync(reportingNode, year, month, economicBasis, Periodicity.Quarterly, ImportFormats.Cashflow);

<a id='view-loaded-data-nodes'></a>
## View loaded Data Nodes

The imported active Data Nodes from <code>Initialization/Systemorph/DataNodes.csv</code> are 

In [ ]:
Test.GetStorage().DataNodeDataBySystemName

<a id='view-loaded-yearly-yield-curve'></a>
## View loaded Yearly Yield Curve

Here the correct Yield Curve is chosen among those imported from <code>Initialization/Systemorph/Data.csv</code> according to the [current period](#year-and-period) and the Aoc Step according the [Aoc Configuration](../DataModel/DataStructure#aoc-configuration). In the following we retrieve the desired identity.

In [ ]:
var aocType = "CL";
var novelty = "C";

In [ ]:
var id = Test.GetIdentity(groupOfContract, aocType, novelty);

In [ ]:
var yearlyYieldCurve = Test.GetStorage().GetYearlyYieldCurve(id, economicBasis);

In [ ]:
yearlyYieldCurve

<a id='view-loaded-nominal-cashflows'></a>
## View loaded Nominal Cashflows

The nominal Cashflows for the current period are loaded together with last period Cashflows for Group of Contract that were already existent. 

The relative files are located in: <code>Files/</code>.

The result is showed in the following list to be compared with the entries of the csv files

In [ ]:
Test.GetStorage().GetRawVariables(groupOfContract)

<br/><br/>
<a id='interest-and-discount-rates'></a>
# Interest and Discount Rates

The Yield Curve ${\text{YC}_i}$ associated to the the [selected period](#year-and-period) and target currency of the Group of Contract Reporting Node is [displayed above](#view-loaded-yearly-yield-curve) and used in this section to calculate the Interest and Discount factors

$$
\text{Interest}_i = \big( 1 + \text{YC}_i \big) ^{\frac{1}{12}} ~,
$$

and

\begin{align}
\text{Discount}_i = \big( 1 + \text{YC}_i \big) ^{-\frac{1}{12}} ~.
\end{align}

In the Scope named *MonthlyRate* the Interest and Discount factors are computed

In [ ]:
var computedMonthlyRates = Test.GetScope<MonthlyRate>( id );

In [ ]:
computedMonthlyRates

## Interest

In the following, the Interest factors computed by the application are recalculated independently according to the formula above in two different ways

In [ ]:
var expectedInterest = yearlyYieldCurve.Select( yc => (1 + yc)^( 1.0 / 12.0) );
expectedInterest

In [ ]:
from yc in yearlyYieldCurve select (1 + yc)^( 1.0 / 12.0 ) 

In [ ]:
computedMonthlyRates.Interest.CheckEquality( expectedInterest ).Should().Be(true);

## Discount

The following recalculates the Discount factors to be compared with the computed values

In [ ]:
var expectedDiscount = yearlyYieldCurve.Select( yc => (1 + yc)^( -1.0 / 12.0) );
expectedDiscount

In [ ]:
computedMonthlyRates.Discount.CheckEquality( expectedDiscount ).Should().Be(true);

## Test method: Get Valid Element

Given the present Yield Curve array has finite *length*, the extrapolation implemented is taking the last element for all successive years. This behavior is given by the function *GetValidElement*. The test in this section assures that constant extrapolation is taken for *index* > *length*

In [ ]:
expectedDiscount.GetValidElement( 120 )

In [ ]:
expectedDiscount.GetValidElement( 120 ).Should().Be( expectedDiscount.Last() );

<br/><br/>
# Cashflows

This section deals with the calculation of the *Cumulated Discounted Cashflows* (CDC) starting from the nominal Cashflows. 

## Choose identity

Each loaded nominal Cashflows is assigned to a so-called *Identity* which uniquely identifies it based on the *Data Node*, *Aoc Type*, and *Novelty*. In the following cells we retrieve the desired Identity:

In [ ]:
var aocType = "CL";
var novelty = "C";

In [ ]:
var id = Test.GetIdentity(groupOfContract, aocType, novelty);

<a id='nominal-cashflow'></a>
## Nominal Cashflow

Consider the two Amount Type *PR* and *NIC*, with Transaction Period being *Beginning of Period* (BOP) and *End of Period* (EOP), respectively. The corresponding nominal Cashflows are deposited in the Scopes named *NominalCashflow* with the Identity provided above, Calculation Type *BE*, and the aforementioned Amount Types. These nominal Cashflows correspond with those [loaded from the csv files](#view-loaded-nominal-cashflows).  

In [ ]:
var amountTypePR = "PR";
var amountTypeNIC = "NIC";
var estimateType = "BE";
var accidentYear = (int?)null;

In [ ]:
var nominalPR = Test.GetScope<NominalCashflow>((id, amountTypePR, estimateType, accidentYear));

In [ ]:
nominalPR

In [ ]:
var nominalNIC = Test.GetScope<NominalCashflow>((id, amountTypeNIC, estimateType, accidentYear));

In [ ]:
nominalNIC

<a id='cumulated-discounted-cashflow'></a>
## Cumulated Discounted Cashflow

The CDC is defined by the following recursive formulas

$$
\text{CDC}_t = \left\{
\begin{array}{cl}
\text{Nominal}_t + \text{CDC}_{t+1} \cdot {\text{Valid Discount}_{\frac{t}{12}}} ~, & \text{if Transaction Period is Beginning of Period} \\
\big( \text{Nominal}_t + \text{CDC}_{t+1} \big) \cdot {\text{Valid Discount}_{\frac{t}{12}}} ~, & \text{if Transaction Period is End of Period}
\end{array}
\right.
$$

where the Transaction Period depends on the given cashflow Amount Type. 

### Beginning of Period

The Amount Type **Premiums** (PR) has Transaction Period **Beginning of Period** (BOP). The corresponding CDC calculated by the application is

In [ ]:
var computedDiscountedPR = Test.GetScope<DiscountedCashflow>((id, amountTypePR, estimateType, accidentYear));

In [ ]:
computedDiscountedPR

In the following cell we independently evaluate the expected discounted cashflow according to the formula above:

In [ ]:
var expectedDiscountedPR = new double[nominalPR.Values.Length];

Enumerable.Range(0, nominalPR.Values.Length).Reverse()
    .ForEach( i => expectedDiscountedPR[i] = nominalPR.Values[i] + expectedDiscountedPR.GetValidElement(i+1) * expectedDiscount.GetValidElement(i/12) );

In [ ]:
expectedDiscountedPR

In [ ]:
(nominalPR.Values, nominalPR.Values.Length)

The expected cumulated discount is compared with the computed values

In [ ]:
computedDiscountedPR.Values.CheckEquality( expectedDiscountedPR )

In [ ]:
computedDiscountedPR.Values.CheckEquality( expectedDiscountedPR ).Should().Be(true);

### End Of Period

The Amount Type **Non Investment Component** (NIC) has Transaction Period **End of Period** (EOP). The corresponding CDC calculated by the application is

In [ ]:
var computedDiscountedNIC = Test.GetScope<DiscountedCashflow>((id, amountTypeNIC, estimateType, accidentYear));

In [ ]:
computedDiscountedNIC

In the following cell we independently evaluate the expected discounted cashflow according to the formula above:

In [ ]:
var expectedDiscountedNIC = new double[nominalNIC.Values.Length];

Enumerable.Range(0, nominalNIC.Values.Length).Reverse()
    .ForEach( i => expectedDiscountedNIC[i] = ( nominalNIC.Values[i] + expectedDiscountedNIC.GetValidElement(i+1) ) * expectedDiscount.GetValidElement(i/12) );

In [ ]:
expectedDiscountedNIC

The expected cumulated discount is compared with the computed values

In [ ]:
computedDiscountedNIC.Values.CheckEquality( expectedDiscountedNIC )

In [ ]:
computedDiscountedNIC.Values.CheckEquality( expectedDiscountedNIC ).Should().Be(true);

<a id='non-performance-risk-adjustment'></a>
## Non Performance Risk Adjustment

In this section we treat the determination of the non-performance risk adjustment, an element offsetting partially future
claims according to the risk that the reinsurance company cannot fulfil the full extent of its obligations.

Firstly, the discounting recursive relation can be written explicitly as

$$
\begin{array}{l}
\text{CDC}_t = \big( \text{Nominal}_t + \text{CDC}_{t+1} \big) \cdot {\text{Valid Discount}_{\frac{t}{12}}} \\
\Rightarrow \text{CDC}_t = \sum_{\tau=t} \big( \text{Valid Discount}_{\frac{t}{12}} \big)^{\tau-t+1} \cdot \text{Nominal}_\tau 
\end{array}
$$

For nominal Cashflows with Amount Type being *Claims*, the risk-adjusted Cumulated Discounted Cashflow 
is assigned the Amount Type **Credit Default Risk** (CDR), 
and is obtained by multiplying the right hand side of the latter formula by the corresponding risk factor

$$
\text{CDC}_t^{\text{Amount Type}=\text{CDR}} = \sum_{\tau=t} \big( \text{Valid Discount}_{\frac{t}{12}} \big)^{\tau-t+1} \cdot \text{Nominal}_\tau^{\text{Claim}} \cdot \big( e^{-\gamma(\tau-t)} -1 \big)
$$

where $\gamma$ is the assumed non-performance probability of the reinsurer to be provided as an [input parameter](../DataModel/DataStructure#credit-default-rate).

This rate is constant for the whole projection period, that is, $\gamma$ is not a function of $t$. 
It effectively amounts to a constant increase in the continuous discount rate by $\gamma$. 

### Explicit vs recursive formula

The explicit formula is tested versus the expected result obtained with the recursive formula

In [ ]:
var expectedDiscountedNICbis = new double[nominalNIC.Values.Length];

//for(int t = 0; t < 12; t++)
//    for(int tau = t; tau < 12; tau++)
//        expectedDiscountedNICbis[t] += nominalNIC.Values[tau] * Math.Pow(expectedDiscount.GetValidElement(t/12), tau-t+1);

Enumerable.Range(0, nominalNIC.Values.Length)
    .ForEach( i => expectedDiscountedNICbis[i] = 
        Enumerable.Range(i, nominalNIC.Values.Length-i).Select( tau => nominalNIC.Values[tau] * Math.Pow(expectedDiscount.GetValidElement(i/12), tau-i+1) ).Sum() );
    
expectedDiscountedNIC.CheckEquality( expectedDiscountedNICbis )

### Risk Adjusted Claims 

The computed CDC with Amount Type being CDR reads

In [ ]:
var computedDiscountedCRD = Test.GetScope<DiscountedCashflow>((id, "CDR", estimateType, accidentYear));

In [ ]:
computedDiscountedCRD

The starting point for the non performance risk adjustment is the computation of the sum of nominal Cashflows with Amount Type **Claims** listed below

In [ ]:
Test.GetStorage().GetClaims()

with the corresponding nominal Cashflows being

In [ ]:
var nominalNIC = Test.GetScope<NominalCashflow>((id, "NIC", estimateType, accidentYear));
nominalNIC

In [ ]:
var nominalICO = Test.GetScope<NominalCashflow>((id, "ICO", estimateType, accidentYear));
nominalICO

In [ ]:
var nominalALE = Test.GetScope<NominalCashflow>((id, "ALE", estimateType, accidentYear));
nominalNIC

In [ ]:
var nominalULE = Test.GetScope<NominalCashflow>((id, "ULE", estimateType, accidentYear));
nominalNIC

In [ ]:
var nominalClaims = nominalICO.Values + nominalNIC.Values + nominalALE.Values + nominalULE.Values;

Secondly, the correct credit risk rate (parameter $\gamma$ in the [formula above](#non-performance-risk-adjustment)) is retrieved from the list of imported [Partner Rating](../DataModel/DataStructure) and [Credit Default Rates](../DataModel/DataStructure) relative to the target [Data Node](#year-and-period)'s Partner

In [ ]:
groupOfContract

In [ ]:
var creditDefaultRate = Test.GetStorage().GetNonPerformanceRiskRate( id );
creditDefaultRate

Lastly, using the [formula above](#non-performance-risk-adjustment), the expected CDR discounted Cashflow can be computed

In [ ]:
var expectedDiscountedCDR = new double[nominalNIC.Values.Length];

Enumerable.Range(0, nominalClaims.Length).ForEach( t => 
    expectedDiscountedCDR[t] = Enumerable.Range(t, nominalClaims.Length-t).Select( tau => 
                                  nominalClaims[tau] * Math.Pow(expectedDiscount.GetValidElement(t/12), tau-t+1) * (Math.Exp(-creditDefaultRate*(tau-t)) - 1)
                               ).Sum() );

expectedDiscountedCDR

Finally, the expected cumulated discount is compared with the computed values

In [ ]:
computedDiscountedCRD.Values.CheckEquality( expectedDiscountedCDR )

In [ ]:
computedDiscountedCRD.Values.CheckEquality( expectedDiscountedCDR ).Should().Be(true);

<br/><br/>
<a id='present-values'></a>
# Present Values

For an implementation of these formulas refer to the [ImportScopeCalculation notebook](../Import/ImportScopeCalculation). 

The company portfolio value in one period is given by the following Analysis of Change AoC steps:
1. Beginning of Period (BoP): starting value of the portfolio as of December last year (in the Year to Date view), 
2. Model Corrections (MC): change of the portfolio value based on the model change,
3. Cash Flow (CF):
4. Interest Accretion (IA): 
5. Assumption Update (AU): 
6. Yield Curve Update (YCU): impact of yield curve update occured during the reporting period,
7. Credit Risk Update (CRU): impact of credit default rate update occured during the reporting period,
8. Experience Variance (EV): value adjustments following insurance related events which took place (e.g. mortality),
9. Combined Liabilities (CL):
10. End of Period (EOP): portfolio value at the end of the current period.

Note: the AoC steps WO, EA and AM are not included in the Present Value calculation, hence their values if inquired is zero.

The Present Value Base (PVB) values are valid for all [Economic Basis](../DataModel/DataStructure#economic-basis) and depends on the corresponding [AoCType](../DataModel/DataStructure#aoc-type):

$$
\text{PVB}_t \big(\text{AoC}\big) = 
\left\{
\begin{array}{cl}
\text{CDC}_t                                                           & \text{if AoC is Beginning of Period (BOP) } \\
-\text{Nominal}_t(\text{Parent AoC})                                   & \text{if AoC is Expected Cashflow (CF) } \\
\text{InterestAccretion}_t                                             & \text{if AoC is Interest Accretion (IA) } \\
\text{CDC}_t(\text{Parent AoC})                                        & \text{if AoC is End of Period (EOP) } \\
\text{CDC}_{t}(\text{current AoC}) - \text{CDC}_{t}(\text{parent AoC}) & \text{otherwise}
\end{array}
\right.
$$

and the Projected Present Value (PPV) reads

$$
\text{PPV} \big(\text{AoC},\text{TS},\text{S}\big) = \left\{
\begin{array}{cll}
\text{PVB}_{S}                        & \text{if AoC Valuation Period is Beginning of Period } & \text{ (e.g. BOP, MC) } \\
\text{PVB}_{S+TS/2 -1}                & \text{if AoC Valuation Period is Mid of Period }       & \text{ (no examples in current data model) } \\
\sum_{i=S}^{S + TS -1}\text{PVB}_{i}  & \text{if AoC Valuation Period is Delta }               & \text{ (e.g. CF, IA) } \\
\text{PVB}_{S + TS}                   & \text{if AoC Valuation Period is End of Period }       & \text{ (e.g. YCU, CRU, EV, CL, EOP) } \\
\end{array}
\right.
$$

where the Shift (S) and Time Step (TS) are derived from the current [Year and Period](#year-and-period)

In [ ]:
var projectionPeriod = 0;

In [ ]:
Test.GetStorage().GetShift(projectionPeriod)

In [ ]:
Test.GetStorage().GetTimeStep(projectionPeriod)

## Load Cashflow Data

In [ ]:
#!eval-notebook "../Initialization/InitSystemorphToMemory"

In [ ]:
Workspace.InitializeFrom(DataSource);

At this point standard Cashflows are loaded.

Initialization of the Test suite: the data loaded from csv files are ready to be used by the Scopes for the calculations

In [ ]:
var Test = await StartCalculatorAsync(reportingNode, year, month, economicBasis, Periodicity.Quarterly, ImportFormats.Cashflow);

## Select Group of Contract

Here, it is possible to select the Group of Contract used to run the next section of this Notebook.

In [ ]:
groupOfContract = "DTR1.1" ;

## Beginning of Period

The computed Present Value for Aoc type Beginning of Period is:

In [ ]:
var idBoP = Test.GetIdentity(groupOfContract, "BOP", "I");
idBoP

In [ ]:
var computedPresentValueBoP_PR = Test.GetScope<PresentValue>((idBoP, "PR", "BE", (int?)null));
computedPresentValueBoP_PR

According to the [formula](#present-values) above, the expected BOP AoCType for PVB is defined to be equal to the discounted cashflows with the same Identity, Amount Type, and Calculation Type, whereas the PPV is the S-th element of the PVB.

In [ ]:
var discountedBoP_PR = Test.GetScope<DiscountedCashflow>((idBoP, "PR", "BE", (int?)null));
discountedBoP_PR

In [ ]:
var expectedPresentValueBoP_PR = discountedBoP_PR.Values;

In the following cell we check whether the expected values match the computed values

In [ ]:
computedPresentValueBoP_PR.Values.CheckEquality( expectedPresentValueBoP_PR )

In [ ]:
computedPresentValueBoP_PR.Values.CheckEquality( expectedPresentValueBoP_PR ).Should().Be(true);

<a id='model-corrections'></a>
## Model Corrections

The computed Present Value for Aoc type Model Corrections is:

In [ ]:
var idMC = Test.GetIdentity(groupOfContract, "MC", "I");
idMC

In [ ]:
var computedPresentValueMC_PR = Test.GetScope<PresentValue>((idMC, "PR", "BE", (int?)null));
computedPresentValueMC_PR

According to the [formula](#present-values) above, for MC AoCType, the PVB is equal to the difference between the CDC of the current AoCType and the CDC of its parent. Additionally, the PPV is the S-th element of the PVB.

In [ ]:
Test.GetScope<ParentAocStep>((idMC, "PR")).Values

In [ ]:
var discountedMC_PR = Test.GetScope<DiscountedCashflow>((idMC, "PR", "BE", (int?)null));
discountedMC_PR

In [ ]:
var expectedPresentValueMC_PR = discountedMC_PR.Values - discountedBoP_PR.Values;

The expected PVB and PPV are compared with the computed results in the following two cells: 

In [ ]:
computedPresentValueMC_PR.Values.CheckEquality( expectedPresentValueMC_PR )

In [ ]:
computedPresentValueMC_PR.Value == expectedPresentValueMC_PR[0]

In [ ]:
computedPresentValueMC_PR.Values.CheckEquality( expectedPresentValueMC_PR ).Should().Be(true);

In [ ]:
computedPresentValueMC_PR.Value.Should().Be( expectedPresentValueMC_PR[0] );

## Cashflow

The computed Present Value for Aoc type Cashflow is:

In [ ]:
var idCF = Test.GetIdentity(groupOfContract, "CF", "I");
idCF

In [ ]:
var computedPresentValueCF_PR = Test.GetScope<PresentValue>((idCF, "PR", "BE", (int?)null));
computedPresentValueCF_PR

According to the [formula](#present-values) above, for CF AoCType, the PVB is equal to the nominal cashflow of its parent AoC step with opposite sign. The PPV is given by the sum of the first 3 elements.

In [ ]:
var cfReferenceAocStep = Test.GetScope<ReferenceAocStep>(idCF).Value;
cfReferenceAocStep

In [ ]:
var cfNominalReference_PR = Test.GetScope<NominalCashflow>((idCF with {AocType = cfReferenceAocStep.AocType}, "PR", "BE", (int?)null));
cfNominalReference_PR

In [ ]:
var expectedPresentValueCF_PR = -1.0 * cfNominalReference_PR.Values;

The expected PVB and PPV are compared with the computed results in the following two cells: 

In [ ]:
computedPresentValueCF_PR.Values.CheckEquality( expectedPresentValueCF_PR )

In [ ]:
computedPresentValueCF_PR.Values[0] + computedPresentValueCF_PR.Values[1] + computedPresentValueCF_PR.Values[2] == computedPresentValueCF_PR.Value

In [ ]:
computedPresentValueCF_PR.Values.CheckEquality( expectedPresentValueCF_PR ).Should().Be(true);

In [ ]:
computedPresentValueCF_PR.Value.Should().Be( computedPresentValueCF_PR.Values[0] + computedPresentValueCF_PR.Values[1] + computedPresentValueCF_PR.Values[2] );

<a id='interest-accretion'></a>
## Interest Accretion

For IA AoCType, the PVB is given by the following relation  

$$
\text{InterestAccretion}_i = \left\{
\begin{array}{cl}
\big(\text{CDC}_i(\text{Parent AoC}) - \text{Nominal}_i(\text{parent AoC}) \big) \cdot \big({\text{Valid Interest}_{\frac{i}{12}}} - 1 \big)~, 
    & \text{if AmountType Transaction Period is Beginning of Period} \\
\text{CDC}_i(\text{parent AoC}) \cdot \big({\text{Valid Interest}_{\frac{i}{12}}} - 1 \big)~, 
    & \text{otherwise}
\end{array}
\right.
$$

while the PPV is given by the sum of the first S elements.

When selecting the Identity, note that the Novelty *C* is not contemplated for Interest Accretion.

In [ ]:
var idIA = Test.GetIdentity(groupOfContract, "IA", "I");
idIA

### Amount Type BOP

The computed Present Value for Amount Type **Premiums** (PR) with Transaction Period **Beginning of Period** (BOP) is

In [ ]:
var computedPresentValueIA_PR = Test.GetScope<PresentValue>((idIA, "PR", "BE", (int?)null));
computedPresentValueIA_PR

The Present Value can be recomputed independently following the aforementioned [equation](#interest-accretion) starting from the interest factors, the nominal and discounted Cashflow of the parent Aoc Step

In [ ]:
var iaComputedMonthlyRates = Test.GetScope<MonthlyRate>( idIA );
iaComputedMonthlyRates

In [ ]:
var iaReferenceAocStep = Test.GetScope<ReferenceAocStep>(idIA).Value;
iaReferenceAocStep

In [ ]:
var nominalIaReference_PR = Test.GetScope<NominalCashflow>((idIA with {AocType = iaReferenceAocStep.AocType}, "PR", "BE", (int?)null));
nominalIaReference_PR

In [ ]:
var discountedIaReference_PR = Test.GetScope<DiscountedCashflow>((idIA with {AocType = iaReferenceAocStep.AocType}, "PR", "BE", (int?)null));
discountedIaReference_PR

In [ ]:
var expectedPresentValueIA_PR = discountedIaReference_PR.Values.Select((val, index) => ( val - nominalIaReference_PR.Values[index]) * ( iaComputedMonthlyRates.Interest.GetValidElement(index/12) -1));

The expected PVB and PPV are compared with the computed results in the following two cells: 

In [ ]:
computedPresentValueIA_PR.Values.CheckEquality( expectedPresentValueIA_PR )

In [ ]:
computedPresentValueIA_PR.Values[0] + computedPresentValueIA_PR.Values[1] + computedPresentValueIA_PR.Values[2] == computedPresentValueIA_PR.Value

In [ ]:
computedPresentValueIA_PR.Values.CheckEquality( expectedPresentValueIA_PR ).Should().Be(true);

In [ ]:
computedPresentValueIA_PR.Value.Should().Be( computedPresentValueIA_PR.Values[0] + computedPresentValueIA_PR.Values[1] + computedPresentValueIA_PR.Values[2] );

### Amount Type EOP

The computed Present Value for Amount Type **Non Investment Component** (NIC) with Transaction Period **End of Period** (EOP) is

In [ ]:
var computedPresentValueIA_NIC = Test.GetScope<PresentValue>((idIA, "NIC", "BE", (int?)null));
computedPresentValueIA_NIC

The Present Value can be recomputed independently following the aforementioned [equation](#interest-accretion) starting from the interest monthly rates, the discounted Cashflow of the parent Aoc Step

In [ ]:
var iaReferenceAocStep = Test.GetScope<ReferenceAocStep>(idIA).Value;
iaReferenceAocStep

In [ ]:
var discountedIaReference_NIC = Test.GetScope<DiscountedCashflow>((idIA with {AocType = iaReferenceAocStep.AocType}, "NIC", "BE", (int?)null));
discountedIaReference_NIC

In [ ]:
var expectedPresentValueIA_NIC = discountedIaReference_NIC.Values.Select((val, index) => val * ( iaComputedMonthlyRates.Interest.GetValidElement(index/12) -1));

The expected PVB and PPV are compared with the computed results in the following two cells: 

In [ ]:
computedPresentValueIA_NIC.Values.CheckEquality( expectedPresentValueIA_NIC )

In [ ]:
computedPresentValueIA_NIC.Values[0] + computedPresentValueIA_NIC.Values[1] + computedPresentValueIA_NIC.Values[2] == computedPresentValueIA_NIC.Value

In [ ]:
computedPresentValueIA_NIC.Values.CheckEquality( expectedPresentValueIA_NIC ).Should().Be(true);

In [ ]:
computedPresentValueIA_NIC.Value.Should().Be( computedPresentValueIA_NIC.Values[0] + computedPresentValueIA_NIC.Values[1] + computedPresentValueIA_NIC.Values[2] );

## Yield Curve Update

The computed Present Value for Aoc type Yield Curve Update is:

In [ ]:
var idYcu = Test.GetIdentity(groupOfContract, "YCU", "I");
idYcu

In [ ]:
var computedPresentValueYCU_PR = Test.GetScope<PresentValue>((idYcu, "PR", "BE", (int?)null));
computedPresentValueYCU_PR

The Present Value can be recomputed independently following the aforementioned [equation](#interest-accretion) starting from the discount factors for beginning of period and for end of period applied to the reference Aoc step, followed by the difference between the CDC obtained with end of period discount factor and the CDC obtained with beginning of period discount factor. 

In [ ]:
var ycReferenceAocStep = Test.GetScope<ReferenceAocStep>(idYcu).Value;
ycReferenceAocStep

In [ ]:
var ycParentAocStep = Test.GetScope<ParentAocStep>((idYcu, "PR")).Values.Single();
ycParentAocStep

In [ ]:
var ycParentId = idYcu with {AocType = ycParentAocStep.AocType, Novelty = ycParentAocStep.Novelty};
ycParentId

Here is the parent AocStep discounted with the beginning of period discount factor.

In [ ]:
var discountedYcParent = Test.GetScope<DiscountedCashflow>((ycParentId, "PR", "BE", (int?)null));
discountedYcParent

Here we independently compute the CDC of the parent AocStep discounted with end of period discount factor.

In [ ]:
var nominalYcParent = Test.GetScope<NominalCashflow>((ycParentId, "PR", "BE", (int?)null));
nominalYcParent

In [ ]:
var ycComputedMonthlyRates = Test.GetScope<MonthlyRate>( idYcu );
ycComputedMonthlyRates

In [ ]:
var ycReferenceDiscountedEop = new double[nominalYcParent.Values.Length];

Enumerable.Range(0, nominalYcParent.Values.Length).Reverse()
    .ForEach( i => ycReferenceDiscountedEop[i] = nominalYcParent.Values[i] + ycReferenceDiscountedEop.GetValidElement(i+1) * ycComputedMonthlyRates.Discount.GetValidElement(i/12) );
ycReferenceDiscountedEop

In [ ]:
var expectedPresentValueYCU_PR = ycReferenceDiscountedEop - discountedYcParent.Values;

In [ ]:
computedPresentValueYCU_PR.Values.CheckEquality( expectedPresentValueYCU_PR )

In [ ]:
computedPresentValueYCU_PR.Value == expectedPresentValueYCU_PR[3]

In [ ]:
computedPresentValueYCU_PR.Values.CheckEquality( expectedPresentValueYCU_PR ).Should().Be(true);

In [ ]:
computedPresentValueYCU_PR.Value.Should().Be( expectedPresentValueYCU_PR[3] );

## Credit Risk Update

This Aoc Step is relevant only in the case of a Group of Reinsurance Contract.

The computed Present Value for Aoc type Credit Risk Update is:

In [ ]:
var idCru = Test.GetIdentity(groupOfContract, "CRU", "I");
idCru

In [ ]:
var computedPresentValueCRU = Test.GetScope<PresentValue>((idCru, "CDR", "BE", (int?)null));
computedPresentValueCRU

The Present Value can be recomputed independently starting as the difference between the CDC obtained with end of period credit default risk rate and the CDC obtained with beginning of period credit default risk rate, both computed using the Nominal cashflow of the reference Aoc Step and the End Of Period discounting factors. 

In [ ]:
var cruReferenceAocStep = Test.GetScope<ReferenceAocStep>(idCru).Value;
cruReferenceAocStep

In [ ]:
var cruReferenceId = idCru with {AocType = cruReferenceAocStep.AocType, Novelty = cruReferenceAocStep.Novelty};
cruReferenceId

In [ ]:
var cruParentAocStep = Test.GetScope<ParentAocStep>((idCru, "CDR")).Values.Single();
cruParentAocStep

In [ ]:
var cruParentId = idCru with {AocType = cruParentAocStep.AocType, Novelty = cruParentAocStep.Novelty};
cruParentId

Here is the parent AocStep discounted with beginning of period credit default risk rate.

In [ ]:
var discountedCruParent = Test.GetScope<DiscountedCashflow>((cruParentId, "CDR", "BE", (int?)null));
discountedCruParent

In [ ]:
var nominalCruReference = Test.GetScope<NominalCashflow>((cruReferenceId, "CDR", "BE", (int?)null)).Values;
nominalCruReference

In [ ]:
var creditDefaultRate = Test.GetStorage().GetNonPerformanceRiskRate( idCru );
creditDefaultRate

In [ ]:
var cruComputedMonthlyRates = Test.GetScope<MonthlyRate>( idCru );
cruComputedMonthlyRates

Lastly, using the [formula above](#non-performance-risk-adjustment), the expected CDR discounted Cashflow can be computed

In [ ]:
var cruExpectedDiscountedCDR = new double[nominalCruReference.Length];

Enumerable.Range(0, nominalCruReference.Length).ForEach( t => 
    cruExpectedDiscountedCDR[t] = Enumerable.Range(t, nominalCruReference.Length-t).Select( tau => 
                                  nominalCruReference[tau] * 
                                  Math.Pow(cruComputedMonthlyRates.Discount.GetValidElement(t/12), tau-t+1) * 
                                 (Math.Exp(-creditDefaultRate*(tau-t)) - 1)
                               ).Sum() );

cruExpectedDiscountedCDR

In [ ]:
var expectedPresentValueCRU = cruExpectedDiscountedCDR - discountedCruParent.Values;

In [ ]:
computedPresentValueCRU.Values.CheckEquality( expectedPresentValueCRU )

In [ ]:
computedPresentValueCRU.Value == expectedPresentValueCRU[3]

In [ ]:
computedPresentValueCRU.Values.CheckEquality( expectedPresentValueCRU ).Should().Be(true);

In [ ]:
computedPresentValueCRU.Value.Should().Be( expectedPresentValueCRU[3] );

## Combined Liabilities

The computed Present Value for Aoc type Combined Liabilities and Amount Type PR is:

In [ ]:
var idCL = Test.GetIdentity(groupOfContract, "CL", "C");
idCL

In [ ]:
var computedPresentValueCL_PR = Test.GetScope<PresentValue>((idCL, "PR", "BE", (int?)null));
computedPresentValueCL_PR

According to the [formula](#present-values) above, for CL AoCType, the PVB is equal to the difference between the CDC of the current AoCType and the CDC of its parent(s). Additionally, the PPV is the $(\text{S}+\text{TS})$-th element of the PVB.

In [ ]:
var clParents = Test.GetScope<ParentAocStep>((idCL,"PR")).Values;
clParents

In [ ]:
var expectedPresentValueCL_PR = Test.GetScope<DiscountedCashflow>((idCL, "PR", "BE", (int?)null)).Values -
                             clParents.Select(aocStep => Test.GetScope<DiscountedCashflow>((idCL with {AocType = aocStep.AocType, Novelty = aocStep.Novelty}, "PR", "BE", (int?)null)).Values).Aggregate();

The expected PVB and PPV are compared with the computed results in the following two cells: 

In [ ]:
computedPresentValueCL_PR.Values.CheckEquality( expectedPresentValueCL_PR )

In [ ]:
computedPresentValueCL_PR.Value == expectedPresentValueCL_PR[3]

In [ ]:
computedPresentValueCL_PR.Values.CheckEquality( expectedPresentValueCL_PR ).Should().Be(true);

In [ ]:
computedPresentValueCL_PR.Value.Should().Be( expectedPresentValueCL_PR[3] );

## End of Period

The computed Present Value for Aoc type End of Period is:

In [ ]:
var idEOP = Test.GetIdentity(groupOfContract, "EOP", "C");
idEOP

In [ ]:
var computedPresentValueEOP_NIC = Test.GetScope<PresentValue>((idEOP, "NIC", "BE", (int?)null));
computedPresentValueEOP_NIC

According to the [formula](#present-values) above, for EOP AoCType, the PVB is equal to the CDC of the parent AoC step, while the PPV is the $(\text{S}+\text{TS})$-th element of the PVB.

In [ ]:
var eopReferenceAocStep = Test.GetScope<ReferenceAocStep>(idEOP).Value;
eopReferenceAocStep

In [ ]:
var discountedCL_NIC = Test.GetScope<DiscountedCashflow>((idEOP with {AocType = eopReferenceAocStep.AocType, Novelty = eopReferenceAocStep.Novelty}, "NIC", "BE", (int?)null));
discountedCL_NIC

In [ ]:
var expectedPresentValueEOP_NIC = discountedCL_NIC.Values;

The expected PVB and PPV are compared with the computed results in the following two cells: 

In [ ]:
computedPresentValueEOP_NIC.Values.CheckEquality( expectedPresentValueEOP_NIC )

In [ ]:
computedPresentValueEOP_NIC.Value == expectedPresentValueEOP_NIC[3]

In [ ]:
computedPresentValueEOP_NIC.Values.CheckEquality( expectedPresentValueEOP_NIC ).Should().Be(true);

In [ ]:
computedPresentValueEOP_NIC.Value.Should().Be( expectedPresentValueEOP_NIC[3] );

## Check for zero PVB values

### Amortization

For AM AoCType, the PVB values are zero, i.e. Amortization is not part of the Present Value computation. In this subsection we check whether the corresponding values are zero accordingly. 

In [ ]:
var idAM = Test.GetIdentity(groupOfContract, "AM", "C");
idAM

In [ ]:
var computedPresentValueAM_PR = Test.GetScope<PresentValue>((idAM, "PR", "BE", (int?)null));
computedPresentValueAM_PR

In [ ]:
var computedPresentValueAM_PR = Test.GetScope<PresentValue>((idAM, "NIC", "BE", (int?)null));
computedPresentValueAM_PR

<br/><br/>
<a id='amortization-factor'></a>
# Amortization Factors

The amortization factors (AMF) are calculated from the corresponding Nominal and Discounted Cashflows with Calculation Type being *Coverage Units* (CU). 
The monthly AMF is

$$
\text{AMF}^{\text{Monthly}}_t = 1 - \frac{ \text{Nominal}^{\text{Coverage Units}}_t } { \text{CDC}^{\text{Coverage Units}}_t } ~,
$$

while the AMF related to the current period is

$$
\text{AMF} = \prod _{t = \text{S}}^{\text{S}+\text{TS}} \text{AMF}^{\text{Monthly}}_t  ~.
$$

where the shift S and Time Step TS for the [current period](#year-and-period) are

In [ ]:
var projectionPeriod = 0;

In [ ]:
var shift = Test.GetStorage().GetShift(projectionPeriod);
shift

In [ ]:
var timeStep = Test.GetStorage().GetTimeStep(projectionPeriod);
timeStep

## Monthly Amortization

The computed monthly and current period Amortization Factors derived from a non-zero [imported](#year-and-period) nominal cashflows and its corresponding discounted are shown in the following

In [ ]:
var idAM = Test.GetIdentity(groupOfContract, "CL", "C");
idAM

In [ ]:
var nominalCashflowAmortizationFactorTest = Test.GetScope<NominalCashflow>( (idAM, default(string), "CU", (int?)null) );
nominalCashflowAmortizationFactorTest

In [ ]:
var discountedCashflowAmortizationFactorTest = Test.GetScope<CoverageUnitCashflow>(idAM);
discountedCashflowAmortizationFactorTest

In [ ]:
var computedMonthlyAmortizationFactor = Test.GetScope<MonthlyAmortizationFactorCashflow>(idAM with {AocType = "AM"});
computedMonthlyAmortizationFactor

In the following cell we independently evaluate the expected monthly amortization factor according to the [formula above](#amortization-factor) and we check whether expected vs computed match

In [ ]:
var expectedMonthlyAmortizationFactor = nominalCashflowAmortizationFactorTest.Values.Zip(discountedCashflowAmortizationFactorTest.Values, 
                                            (nominal, discountedCumulated) => 1 - nominal / discountedCumulated ).ToArray();

In [ ]:
expectedMonthlyAmortizationFactor.RepeatOnce()

In [ ]:
computedMonthlyAmortizationFactor.MonthlyAmortizationFactors.CheckEquality( expectedMonthlyAmortizationFactor )

In [ ]:
computedMonthlyAmortizationFactor.MonthlyAmortizationFactors.CheckEquality( expectedMonthlyAmortizationFactor ).Should().Be(true);

## Current Period

The computed current period amortization factor is

In [ ]:
var computedCurrentPeriodAmortizationFactor = Test.GetScope<CurrentPeriodAmortizationFactor>(idAM with {AocType = "AM"});
computedCurrentPeriodAmortizationFactor

Conversely, the current period Amortization factor can be independently computed using the [formula above](#amortization-factor) as

In [ ]:
var expectedCurrentPeriodAmortizationFactor = expectedMonthlyAmortizationFactor.Skip(shift).Take(timeStep).Aggregate(1d, (x, y) => x * y);

In [ ]:
expectedCurrentPeriodAmortizationFactor

In [ ]:
computedCurrentPeriodAmortizationFactor.Value == expectedCurrentPeriodAmortizationFactor

In [ ]:
computedCurrentPeriodAmortizationFactor.Value.Should().Be( expectedCurrentPeriodAmortizationFactor );

<br/><br/>
<a id='persist-results'></a>
# Results persisted

The last step during the Cashflow import is to save, namely *persist* the Present Values computed with *locked* (L) and *current* (C) economic basis, and calculation type being *Best Estimate* (BE). 

These data is the starting point of the [Contractual Service Margin calculation]("SpecificationsCSM").